# Text2Cypher, Graph Retrieval Arguments Generation(RAG), Graph RAG and Graph+Vector RAG

To enable Cognitive intelligence App towards private data, RAG + LLM and Knowledge Graph are state-of-the-art approaches.

In this demo, we will explain know-how of four types of approaches and compare the trade-off and performance among them.

| QueryEngine | Knowledge Graph query engine                                 | Graph RAG query engine                                       | Vector RAG query engine                                      | Graph Vector RAG query engine                                |
| ----------- | ------------------------------------------------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ |
| Mechanism   | 1. **Text-to-GraphQuery** based on KG<br />2. Query KG with the result<br />3. Answer synthesis based on query result | 1. Get related entities of the question<br />2. Get n-depth **SubGraphs** of related entities from KG<br />3. Answer synthesis based on related SubGraphs | 1. Create embedding of question<br />2. Semantic search **top-k related doc chunks**<br />3. Answer synthesis based on related doc chunks | 1. Do retrieval as Vector and Graph RAG <br />2. Answer synthesis based on **both related chunks and SubGraphs** |

## Background, RAG

Below digrams are showing how RAG works:

```
                  RAG with Llama Index
                  ┌────┬────┬────┬────┐                  
                  │ 1  │ 2  │ 3  │ 4  │                  
                  ├────┴────┴────┴────┤                  
                  │  Docs/Knowledge   │                  
┌───────┐         │        ...        │       ┌─────────┐
│       │         ├────┬────┬────┬────┤       │         │
│       │         │ 95 │ 96 │    │    │       │         │
│       │         └────┴────┴────┴────┘       │         │
│ User  │─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─▶   LLM   │
│       │                                     │         │
│       │                                     │         │
└───────┘    ┌ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ┐  └─────────┘
    │          ┌──────────────────────────┐        ▲     
    └────────┼▶│  Tell me ....., please   │├───────┘     
               └──────────────────────────┘              
             │ ┌────┐ ┌────┐               │             
               │ 3  │ │ 96 │                             
             │ └────┘ └────┘               │             
              ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ 
```

In VectorDB based RAG, we create embeddings of each node(chunk), and find TopK related ones towards a given question during the query. In the above diagram, nodes `3` and `96` were fetched as the TopK related nodes, used to help answer the user query. 

## Background Graph RAG

In Graph RAG, we will extract relationships between entities, representing concise facts from each node. It would look something like this:

```
Node Split and Embedding

┌────┬────┬────┬────┐
│ 1  │ 2  │ 3  │ 4  │
├────┴────┴────┴────┤
│  Docs/Knowledge   │
│        ...        │
├────┬────┬────┬────┤
│ 95 │ 96 │    │    │
└────┴────┴────┴────┘
```

Then, if we zoomed in of it:

```
       Node Split and Embedding, with Knowledge Graph being extracted

┌──────────────────┬──────────────────┬──────────────────┬──────────────────┐
│ .─.       .─.    │  .─.       .─.   │            .─.   │  .─.       .─.   │
│( x )─────▶ y )   │ ( x )─────▶ a )  │           ( j )  │ ( m )◀────( x )  │
│ `▲'       `─'    │  `─'       `─'   │            `─'   │  `─'       `─'   │
│  │     1         │        2         │        3    │    │        4         │
│ .─.              │                  │            .▼.   │                  │
│( z )─────────────┼──────────────────┼──────────▶( i )─┐│                  │
│ `◀────┐          │                  │            `─'  ││                  │
├───────┼──────────┴──────────────────┴─────────────────┼┴──────────────────┤
│       │                      Docs/Knowledge           │                   │
│       │                            ...                │                   │
│       │                                               │                   │
├───────┼──────────┬──────────────────┬─────────────────┼┬──────────────────┤
│  .─.  └──────.   │  .─.             │                 ││  .─.             │
│ ( x ◀─────( b )  │ ( x )            │                 └┼▶( n )            │
│  `─'       `─'   │  `─'             │                  │  `─'             │
│        95   │    │   │    96        │                  │   │    98        │
│            .▼.   │  .▼.             │                  │   ▼              │
│           ( c )  │ ( d )            │                  │  .─.             │
│            `─'   │  `─'             │                  │ ( x )            │
└──────────────────┴──────────────────┴──────────────────┴──`─'─────────────┘
```

Where, knowledge, the more granular spliting and information with higher density, optionally multi-hop of `x -> y`, `i -> j -> z -> x` etc... across many more nodes(chunks) than K(in TopK search) could be inlucded in Retrievers. And we believe there are cases that this additional work matters.

But how/how well exactly does it work? Let's see in this demo.

# 1. Preparation

## 1.1 Prepare for LLM

In [1]:
# For OpenAI

import os

# os.environ["OPENAI_API_KEY"] = "INSERT YOUR KEY"

import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output

from llama_index.core import (
    KnowledgeGraphIndex,
    VectorStoreIndex,
    ServiceContext,
    SimpleDirectoryReader,
    StorageContext,
    PromptTemplate
)
from llama_index.graph_stores.nebula import NebulaGraphStore


from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from IPython.display import Markdown, display

from finllmqa.api.core import LLM_API_URL

from llama_index.core import Settings

llm = OpenAI(model="gpt-3.5-turbo", api_base=LLM_API_URL, api_key='null')
embed_model = OpenAIEmbedding(api_base=LLM_API_URL, api_key='null')

Settings.llm = llm
Settings.embed_model = embed_model

# # define LLM
# # NOTE: at the time of demo, text-davinci-002 did not have rate-limit errors
# llm = OpenAI(temperature=0, model="gpt-3.5-turbo", api_base=LLM_API_URL, api_key='null')
# embed_model = OpenAIEmbedding(api_base=LLM_API_URL, api_key='null')
# service_context = ServiceContext.from_defaults(llm=llm, embed_model=chunk_size=512)

In [2]:
# For Azure OpenAI

import os
import json
import openai
from langchain.embeddings import OpenAIEmbeddings
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    KnowledgeGraphIndex,
    ServiceContext
)
from llama_index.core import set_global_service_context

from llama_index.core.storage.storage_context import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore

import logging
import sys

from IPython.display import Markdown, display

from finllmqa.api.core import LLM_API_URL

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

openai.api_type = "azure"
openai.api_base = LLM_API_URL
openai.api_version = "2024-03-01"
# os.environ["OPENAI_API_KEY"] = "youcannottellanyone"
# openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = 'null'

llm = AzureOpenAI(
    engine="<foo-bar-deployment>",
    temperature=0,
    openai_api_version=openai.api_version,
    model_kwargs={
        "api_key": openai.api_key,
        "api_base": openai.api_base,
        "api_type": openai.api_type,
        "api_version": openai.api_version,
    },
)

# You need to deploy your own embedding model as well as your own chat completion model
embedding_llm = LangchainEmbedding(
    OpenAIEmbeddings(
        model="text-embedding-ada-002",
        deployment="<foo-bar-deployment>",
        openai_api_key=openai.api_key,
        openai_api_base=openai.api_base,
        openai_api_type=openai.api_type,
        openai_api_version=openai.api_version,
    ),
    embed_batch_size=1,
)

# service_context = ServiceContext.from_defaults(
#     llm=llm,
#     embed_model=embedding_llm,
# )

# set_global_service_context(service_context)

E:\anaconda\app\envs\chatglm3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
E:\anaconda\app\envs\chatglm3\lib\site-packages\langchain_community\embeddings\openai.py:336: UserWarning: If you have openai>=1.0.0 installed and are using Azure, please use the `AzureOpenAIEmbeddings` class.
  warnings.warn(
D:\Personal\Temp\ipykernel_12480\787946910.py:64: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(

In [3]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embedding_llm
Settings.chunk_size = 512

## 1.2. Prepare for NebulaGraph as Graph Store


❗Access NebulaGraph Console to **create space** and **graph schema**

```sql
CREATE SPACE guardians(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);
:sleep 10;
USE guardians;
CREATE TAG entity(name string);
CREATE EDGE relationship(relationship string);
:sleep 10;
CREATE TAG INDEX entity_index ON entity(name(256));
```

In [2]:
# %pip install nebula3-python ipython-ngql

os.environ['NEBULA_USER'] = "root"
os.environ['NEBULA_PASSWORD'] = "nebula" # default password
os.environ['NEBULA_ADDRESS'] = "127.0.0.1:9669" # assumed we have NebulaGraph installed locally

# space_name = "guardians"
space_name = "financial_books"
# edge_types, rel_prop_names = ["relationship"], ["relationship"] # default, could be omit if create from an empty kg
# tags = ["entity"] # default, could be omit if create from an empty kg
edge_types, rel_prop_names = ["relationship"], ["name"] # default, could be omit if create from an empty kg
tags = ["book"] # default, could be omit if create from an empty kg

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)

## 2. Build the Knowledge Graph

In our demo, the Knowledge Graph was created with LLM.

We simply do so leveragint the `KnowledgeGraphIndex` from LlamaIndex, when creating it, Triplets will be extracted with LLM and evantually persisted into `NebulaGraphStore`.

### 2.1 Preprocess Data

We will download and preprecess data from:
    https://en.wikipedia.org/wiki/Guardians_of_the_Galaxy_Vol._3

In [4]:
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()

documents = loader.load_data(pages=['Guardians of the Galaxy Vol. 3'], auto_suggest=False)

In [24]:
storage_context = StorageContext.from_defaults(graph_store=graph_store)
documents = SimpleDirectoryReader(input_files=[f"books/宏观经济学.pdf"]).load_data()

### 2.2 Extract Triplets and Save to NebulaGraph

This call will take some time, it'll extract entities and relationships and store them into NebulaGraph

In [4]:
kg_prompt_template = """
    下面提供了一些文本。根据文本，提取最多 {max_knowledge_triplets} 个三元组的知识，形式为（主语，谓语，宾语），避开停用词。
    ---------------------
    示例：
    文本：小红是小明的母亲.
    三元组：
    （小红，是母亲，小明）
    文本：瑞幸是2017年在厦门创立的咖啡店。
    三元组：
    （瑞幸，是，咖啡店）
    （瑞幸，创立于，厦门）
    （瑞幸，创立于，2017）
    ---------------------
    文本：{text}
    三元组："""
kg_prompt_template = PromptTemplate(kg_prompt_template)
kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
#     service_context=service_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    include_embeddings=True,
    kg_triple_extract_template=kg_prompt_template
)

INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
IN

## 3 Create VectorStoreIndex for RAG

To compare with/work together with VectorDB based RAG, let's also create a `VectorStoreIndex`.

During the creation, same data source will be split into chunks and embedding of them will be created, during the RAG query time, the top-k related embeddings will be vector-searched with the embedding of the question.

```
                  RAG with Llama Index
                  ┌────┬────┬────┬────┐                  
                  │ 1  │ 2  │ 3  │ 4  │                  
                  ├────┴────┴────┴────┤                  
                  │  Docs/Knowledge   │                  
┌───────┐         │        ...        │       ┌─────────┐
│       │         ├────┬────┬────┬────┤       │         │
│       │         │ 95 │ 96 │    │    │       │         │
│       │         └────┴────┴────┴────┘       │         │
│ User  │─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─▶   LLM   │
│       │                                     │         │
│       │                                     │         │
└───────┘    ┌ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ┐  └─────────┘
    │          ┌──────────────────────────┐        ▲     
    └────────┼▶│  Tell me ....., please   │├───────┘     
               └──────────────────────────┘              
             │ ┌────┐ ┌────┐               │             
               │ 3  │ │ 96 │                             
             │ └────┘ └────┘               │             
              ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ 
```

In Llama Index, this could be done with oneline of code:

In [8]:
vector_index = VectorStoreIndex.from_documents(
    documents,
#     service_context=service_context
)

INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"


## 4. Persist and Load from disk Llama Indexes(Optional)

Both the `KnowledgeGraphIndex` and `VectorStoreIndex` will be created only once, afterwards, we could persist their in-memory context to enable their reuse from disk anytime.

#### Persist

```python
# persist KG Index(Only MetaData will be persisted, KG is in NebulaGraph)
kg_index.storage_context.persist(persist_dir='./storage_graph')

# persist Vector Index
vector_index.storage_context.persist(persist_dir='./storage_vector')

```

Then the files are created:

```bash
$ ls -l ./storage_*

storage_graph:
total 6384
-rw-r--r--@ 1 weyl  staff    44008 Jul 14 11:06 docstore.json
-rw-r--r--@ 1 weyl  staff  3219385 Jul 14 11:06 index_store.json
-rw-r--r--@ 1 weyl  staff       51 Jul 14 11:06 vector_store.json

storage_vector:
total 712
-rw-r--r--@ 1 weyl  staff   44008 Jul 14 11:06 docstore.json
-rw-r--r--@ 1 weyl  staff      18 Jul 14 11:06 graph_store.json
-rw-r--r--@ 1 weyl  staff    1003 Jul 14 11:06 index_store.json
-rw-r--r--@ 1 weyl  staff  311028 Jul 14 11:06 vector_store.json
```

#### Restore

So we could restore the index from disk like:

```python
from llama_index import load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir='./storage_graph', graph_store=graph_store)
kg_index = load_index_from_storage(
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    include_embeddings=True,
)

storage_context_vector = StorageContext.from_defaults(persist_dir='./storage_vector')
vector_index = load_index_from_storage(
    service_context=service_context,
    storage_context=storage_context_vector
)
```

In [3]:
# vector_index.storage_context.persist(persist_dir='./storage_vector')

from llama_index.core import load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir='./storage_graph', graph_store=graph_store)
kg_index = load_index_from_storage(
    storage_context=storage_context,
#     service_context=service_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    include_embeddings=True,
)

storage_context_vector = StorageContext.from_defaults(persist_dir='./storage_vector')
vector_index = load_index_from_storage(
#     service_context=service_context,
    storage_context=storage_context_vector
)

INFO:llama_index.core.indices.loading:Loading all indices.
INFO:llama_index.core.indices.loading:Loading all indices.


## 5. Prepare for different query approaches

We will do 4 types of query approaches with LLM, KG, VectorDB:

| QueryEngine | Knowledge Graph query engine                                 | Graph RAG query engine                                       | Vector RAG query engine                                      | Graph Vector RAG query engine                                |
| ----------- | ------------------------------------------------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ |
| Mechanism   | 1. **Text-to-GraphQuery** based on KG<br />2. Query KG with the result<br />3. Answer synthesis based on query result | 1. Get related entities of the question<br />2. Get n-depth **SubGraphs** of related entities from KG<br />3. Answer synthesis based on related SubGraphs | 1. Create embedding of question<br />2. Semantic search **top-k related doc chunks**<br />3. Answer synthesis based on related doc chunks | 1. Do retrieval as Vector and Graph RAG <br />2. Answer synthesis based on **both related chunks and SubGraphs** |


### 5.1 text-to-NebulaGraphCypher

Text-to-NebulaGraphCypher approach Translate task/question into a Graph Cypher Query, and answer based on its query result.

In [5]:
from llama_index.core.query_engine import KnowledgeGraphQueryEngine

from llama_index.core.storage.storage_context import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore

# llm = OpenAI(model="gpt-3.5-turbo", api_base=LLM_API_URL, api_key='null')

nl2kg_query_engine = KnowledgeGraphQueryEngine(
    storage_context=storage_context,
#     service_context=service_context,
#     llm=llm,
    verbose=True,
    g
)

In [22]:
nl2kg_query_engine.get_prompts()

{'graph_query_synthesis_prompt': None,
 'graph_response_answer_prompt': PromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['query_str', 'kg_query_str', 'kg_response_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='\nThe original question is given below.\nThis question has been translated into a Graph Database query.\nBoth the Graph query and the response are given below.\nGiven the Graph Query response, synthesise a response to the original question.\n\nOriginal question: {query_str}\nGraph query: {kg_query_str}\nGraph response: {kg_response_str}\nResponse:\n'),
 'response_synthesizer:text_qa_template': SelectorPromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_A

### 5.2 Graph RAG query engine

Graph RAG takes SubGraphs related to entities of the task/question as Context.

```
           Graph + Vector RAG with Llama Index
                  ┌────┬────┬────┬────┐                  
                  │ 1  │ 2  │ 3  │ 4  │                  
                  ├────┴────┴────┴────┤                  
                  │  Docs/Knowledge   │                  
┌───────┐         │        ...        │       ┌─────────┐
│       │         ├────┬────┬────┬────┤       │         │
│       │         │ 95 │ 96 │    │    │       │         │
│       │         └────┴────┴────┴────┘       │         │
│ User  │─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─▶   LLM   │
│       │                                     │         │
│       │                                     │         │
└───────┘    ┌ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ┐  └─────────┘
    │          ┌──────────────────────────┐        ▲     
    └────────┼▶│  Tell me about x, please │├───────┘     
               └──────────────────────────┘              
             │ Below are knowledge about x │             
               x->y<-z,x->h->i, m<-n,...                            
             │ Please answer based on them │             
              ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ 
```

In [6]:
kg_rag_query_engine = kg_index.as_query_engine(
    include_text=False,
    retriever_mode="keyword",
    response_mode="tree_summarize",
)

### 5.3 Vector RAG query engine

Vector RAG is the common approach to find topK semantic related doc chunks as context to synthesize the answer.

In [7]:
vector_rag_query_engine = vector_index.as_query_engine()

### 5.4 Graph+Vector RAG query engine

This is a combined Graph+Vector Based RAG, where we will retrieve both VectorDB and KG SubGraphs as the context, for synthesis of the answer.

```
           Graph + Vector RAG with Llama Index
                  ┌────┬────┬────┬────┐                  
                  │ 1  │ 2  │ 3  │ 4  │                  
                  ├────┴────┴────┴────┤                  
                  │  Docs/Knowledge   │                  
┌───────┐         │        ...        │       ┌─────────┐
│       │         ├────┬────┬────┬────┤       │         │
│       │         │ 95 │ 96 │    │    │       │         │
│       │         └────┴────┴────┴────┘       │         │
│ User  │─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─▶   LLM   │
│       │                                     │         │
│       │                                     │         │
└───────┘    ┌ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ┐  └─────────┘
    │          ┌──────────────────────────┐        ▲     
    └────────┼▶│  Tell me ....., please   │├───────┘     
               └──────────────────────────┘              
             │ ┌────┐┌────┐               │             
               │ 3  ││ 96 │ x->y<-z,x->h...                            
             │ └────┘└────┘               │             
              ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ 
```

To implement that in Llama Index, we create a `CustomRetriever` to comebine the two: 

In [8]:
# import QueryBundle
from llama_index.core import QueryBundle

# import NodeWithScore
from llama_index.core.schema import NodeWithScore

# Retrievers
from llama_index.core.retrievers import BaseRetriever, VectorIndexRetriever, KGTableRetriever

from typing import List


class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both Vector search and Knowledge Graph search"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        kg_retriever: KGTableRetriever,
        mode: str = "OR",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._kg_retriever = kg_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        kg_nodes = self._kg_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        kg_ids = {n.node.node_id for n in kg_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in kg_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(kg_ids)
        else:
            retrieve_ids = vector_ids.union(kg_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

Next, we will create instances of the Vector and KG retrievers, which will be used in the instantiation of the Custom Retriever.

In [9]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

# create custom retriever
vector_retriever = VectorIndexRetriever(index=vector_index)
kg_retriever = KGTableRetriever(
    index=kg_index, retriever_mode="keyword", include_text=False
)
custom_retriever = CustomRetriever(vector_retriever, kg_retriever)

# create response synthesizer
response_synthesizer = get_response_synthesizer(
#     service_context=service_context,
    response_mode="tree_summarize",
)

And the query engine:

In [10]:
graph_vector_rag_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer
)

## 6. Query with all the Engines

### 6.1 Text-to-GraphQuery

In [21]:
response_nl2kg = nl2kg_query_engine.query("什么是经济学十大原理.")


display(Markdown(f"<b>{response_nl2kg}</b>"))

# Cypher:

print("Cypher Query:")

graph_query = nl2kg_query_engine.generate_query(
    "什么是经济学十大原理",
)
graph_query = graph_query.replace("WHERE", "\n  WHERE").replace("RETURN", "\nRETURN")

display(
    Markdown(
        f"""
```cypher
{graph_query}
```
"""
    )
)

AttributeError: 'NoneType' object has no attribute 'kwargs'

### 6.2 Graph RAG

In [11]:
response_graph_rag = kg_rag_query_engine.query("什么是经济学十大原理")

display(Markdown(f"<b>{response_graph_rag}</b>"))

INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"


<b>根据提供的知识序列，经济学十大原理是指一系列关于经济学的原理和概念。这些原理包括了许多重要的经济学概念，如稀缺性、需求、供应、生产、消费、投资、分配、交换、货币、利率和增长等。通过学习和理解这些原理，人们可以更好地理解经济运行的规律和机制，从而做出更明智的决策。</b>

### 6.3 Vector RAG

In [12]:
response_vector_rag = vector_rag_query_engine.query("什么是经济学十大原理")

display(Markdown(f"<b>{response_vector_rag}</b>"))

INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"


<b>经济学十大原理是《经济学原理》这本书中的核心概念。它们包括：

1. 人们面临交替关系：为了得到一种东西，我们必须放弃另一种东西。
2. 某种东西的成本是为了得到它而放弃的东西。
3. 理性人考虑边际量：人们在做出决策时会考虑边缘效益。
4. 人们会对激励作出反应：人们的决策受到激励的影响。
5. 人们如何相互交易：交易是经济活动的基本方式。
6. 贸易能使每个人状况更好：贸易能够提高参与者的福利水平。
7. 市场通常是组织经济活动的一种好方法：市场能够有效地分配资源。
8. 政府有时可以改善市场结果：政府在某些情况下可以采取措施来优化市场效果。
9. 整体经济如何运行：经济体系中的各种因素共同决定了整体经济的表现。
10. 一国的生活水平取决于它生产物品与劳务的能力：一个国家的经济实力决定了民众的生活水平。</b>

In [20]:
display(
    Markdown(
        llm.complete(f"""
Compare the two QA result on "什么是经济学十大原理", list the differences between them, to help evalute them. Output in markdown table.

Result from Graph: {response_graph_rag}
---
Result from Vector: {response_vector_rag}

"""
           ).text
    )
)

INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"


|    来源    |     内容                                       |
| :---------: | :--------------------------------------------- |
| 图形结果    | 经济学十大原理是指一系列关于经济学的原理和概念。这些原理包括了许多重要的经济学概念，如稀缺性、需求、供应、生产、消费、投资、分配、交换、货币、利率和增长等。 |
| 向量结果    | 经济学十大原理是《经济学原理》这本书中的核心概念。它们包括：1. 人们面临交替关系：为了得到一种东西，我们必须放弃另一种东西。2. 某种东西的成本是为了得到它而放弃的东西。3. 理性人考虑边际量：人们在做出决策时会考虑边缘效益。4. 人们会对激励作出反应：人们的决策受到激励的影响。5. 人们如何相互交易：交易是经济活动的基本方式。6. 贸易能使每个人状况更好：贸易能够提高参与者的福利水平。7. 市场通常是组织经济活动的一种好方法：市场能够有效地分配资源。8. 政府有时可以改善市场结果：政府在某些情况下可以采取措施来优化市场效果。9. 整体经济如何运行：经济体系中的各种因素共同决定了整体经济的表现。10. 一国的生活水平取决于它生产物品与劳务的能力：一个国家的经济实力决定了民众的生活水平。 |

根据上述比较，两个结果在概述经济学十大原理方面存在一些差异。图形结果提供了更具体的内容，列出了十大原理中涉及的一些重要概念。相比之下，向量结果提供了《经济学原理》这本书的核心概念，突出了它们作为书籍的主要内容。因此，两个结果在描述经济学十大原理时各有侧重，可以根据需要选择使用其中一个或两个结果。

### 6.4 Graph + Vector RAG

In [16]:
response_graph_vector_rag = graph_vector_rag_query_engine.query("什么是经济学十大原理")

display(Markdown(f"<b>{response_graph_vector_rag}</b>"))

INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://gemini2.sufe.edu.cn:27282/v1/chat/completions "HTTP/1.1 200 OK"


<b>经济学十大原理是经济学的基本概念和原则，它们构成了经济学的核心框架。这些原理包括：人们如何作出决策、某种东西的成本是为了得到它而放弃的东西、理性人考虑边际量、人们会对激励作出反应、人们如何相互交易、贸易能使每个人状况更好、市场通常是组织经济活动的一种好方法、政府有时可以改善市场结果、整体经济如何运行以及一国的生活水平取决于它生产物品与劳务的能力。这些原理在经济学的各个领域中都有重要的应用，包括微观经济学和宏观经济学。通过理解和运用这些原理，我们可以更好地理解经济现象和做出明智的决策。</b>

## 7. Conclusion

### 7.1 Overall Comparision

Let's compare the results of them.

First check the information that were coverred by different approaches:

In [43]:
display(
    Markdown(
        llm(f"""
Compare the QA results on "Tell me about Peter Quill.", list the knowledge facts between them, to help evalute them. Output in markdown table.

Result text2GraphQuery: {response_nl2kg}
---
Result Graph: {response_graph_rag}
---
Result Vector: {response_vector_rag}
---
Result Graph+Vector: {response_graph_vector_rag}
---

"""
           )
    )
)


| Knowledge Fact | text2GraphQuery | Graph | Vector | Graph+Vector |
| -------------- | --------------- | ----- | ------ | ----------- |
| Member of Guardians of the Galaxy | Yes | Yes | No | Yes |
| Leader of Guardians of the Galaxy | No | Yes | No | Yes |
| Portrayed by Chris Pratt | No | Yes | No | Yes |
| Wrote and directed first movie | No | Yes | No | Yes |
| Foul language | No | Yes | No | Yes |
| Half-human, half-Celestial | No | No | Yes | Yes |
| Abducted from Earth as a child | No | No | Yes | Yes |
| Raised by a group of alien thieves and smugglers | No | No | Yes | Yes |
| In a "state of depression" | No | No | Yes | No |

**Conclusion**

- The pure **KG**(both text2GraphQuery and Graph RAG) comes with **concise** results, and much **lower cost**(for cost comparision see our previous result [here](https://gpt-index.readthedocs.io/en/latest/examples/index_structs/knowledge_graph/KnowledgeGraphIndex_vs_VectorStoreIndex_vs_CustomIndex_combined.html#comparison-of-results) )
- The **Graph+Vector** RAG could be more **comprehensive** in case the question envolves knowledge that's fine-grained **spread** across more chunks than top-K searching.


| QueryEngine | Knowledge Graph query engine                                 | Graph RAG query engine                                       | Vector RAG query engine                                      | Graph Vector RAG query engine                                |
| ----------- | ------------------------------------------------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ |
| Mechanism   | 1. **Text-to-GraphQuery** based on KG<br />2. Query KG with the result<br />3. Answer synthesis based on query result | 1. Get related entities of the question<br />2. Get n-depth **SubGraphs** of related entities from KG<br />3. Answer synthesis based on related SubGraphs | 1. Create embedding of question<br />2. Semantic search **top-k related doc chunks**<br />3. Answer synthesis based on related doc chunks | 1. Do retrieval as Vector and Graph RAG <br />2. Answer synthesis based on **both related chunks and SubGraphs** |
| Performance | Concise                                                      | Concise                                                      | Fruitful                                                     | Fruitful, could be more comprehensive                        |
| Cost        | Low                                                          | Low                                                          | High                                                         | High                                                         |


### 7.2 Text2GraphQuery vs Graph RAG

In **Text2GraphQuery**, we leverage the LLM to compose a Graph Query that's trying to provide the answer in the `RETURN` fields. While, on the other hands, the **Graph RAG** find all related knowledges as the context.

So when the answer by nature refers to small piece of information as answer, Text2GraphQuery could do the job in the best, and in other cases, the Graph RAG could be better:

In [31]:
display(
    Markdown(
        llm(f"""
Compare the two QA result on "Tell me about Peter Quill.", list the differences between them, to help evalute them. Output in markdown table.

Result from text-to-Graph: {response_nl2kg}
---
Result from Graph RAG: {response_graph_rag}

"""
           )
    )
)


| Feature | Text-to-Graph | Graph RAG |
| --- | --- | --- |
| Superhero Team | Guardians of the Galaxy | Guardians of the Galaxy |
| Movie | N/A | 2014 and 2017 |
| Role | N/A | Portrayed by Chris Pratt |
| Writing/Directing | N/A | Wrote and Directed the first movie |
| Character Trait | N/A | Foul language, often using the word "fuck" |

We could visulize the knowledge context that were coverred during the two approach, we could see their difference: 

In [58]:
graph_text2cypher = _

In [55]:
%%ngql
MATCH path_0=(p:`entity`)-[*1..2]-()
  WHERE p.`entity`.`name` == 'Peter Quill' 
RETURN path_0

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)


,path_0
0,"(""Peter Quill"" :entity{name: ""Peter Quill""})-[..."
1,"(""Peter Quill"" :entity{name: ""Peter Quill""})<-..."
2,"(""Peter Quill"" :entity{name: ""Peter Quill""})-[..."
3,"(""Peter Quill"" :entity{name: ""Peter Quill""})-[..."
4,"(""Peter Quill"" :entity{name: ""Peter Quill""})-[..."
5,"(""Peter Quill"" :entity{name: ""Peter Quill""})-[..."


In [56]:
graph_rag = _

In [71]:
text2Cypher = """
<iframe
    src="https://www.siwei.io/demo-dumps/kg-llm/nebulagraph_draw_nl2cypher.html"
    width=450
    height=400>
</iframe>
"""
graphRAG = """
<iframe
    src="https://www.siwei.io/demo-dumps/kg-llm/nebulagraph_draw_rag.html"
    width=450
    height=400>
</iframe>
"""

table = f"""
<table>
    <tr>
        <th>Text2Cypher Traversed knowledge</th>
        <th>Graph Rag Traversed knowledge</th>
    </tr>
    <tr>
        <td>{text2Cypher}</td>
        <td>{graphRAG}</td>
    </tr>
</table>
"""

display(HTML(table))


Text2Cypher Traversed knowledge,Graph Rag Traversed knowledge
,


**Conclusion**

For those tasks:

- Potentially cares more relationed knowledge
- Schema of the KG is sophisticated to be hard for text2cypher to express the task
- KG quality isn't good enough
- Multiple "starting entities" are involved

Graph RAG could be a better approach to start with.